In [3]:
# 安装 pip install msrestazure
# 安装 pip install kubeflow-fairing #装的好慢。。安装了一堆烂七八糟的包。。

from kubeflow import fairing   
from kubeflow.fairing.kubernetes.utils import mounting_pvc
import logging
from kubeflow.fairing import constants
from kubeflow.fairing.preprocessors import base as base_preprocessor
DOCKER_REGISTRY = 'k3d-myregistry.localhost:5000' # 注意nsloopup  k3d-myregistry.localhost 192.168.128.2 端口5000是k3d配置的在host上。。
constants.constants.KANIKO_IMAGE = "aiotceo/kaniko-executor:v1.6.0"

output_map =  {
    "/home/jovyan/project/pytorch-bert/kaniko/dockerfile": "Dockerfile",
    "/home/jovyan/project/pytorch-bert/kaniko": "kaniko"
}

preprocessor = base_preprocessor.BasePreProcessor(
    command=["python"], # The base class will set this.
    input_files=[],
    path_prefix="/app", # irrelevant since we aren't preprocessing any files
    output_map=output_map)

preprocessor.preprocess()



from kubeflow.fairing.builders import cluster


from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

minio_endpoint = "http://minio-service.kubeflow:9000/"
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"
# mc config host add minio http://minio-service.kubeflow:9000 minio minio123

minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

cluster_builder = cluster.cluster.ClusterBuilder(registry=DOCKER_REGISTRY,
                                                 base_image="", # base_image is set in the Dockerfile
                                                 preprocessor=preprocessor,
                                                 image_name="mnist",
                                                 dockerfile_path="Dockerfile",
                                                 context_source=minio_context_source)

In [4]:
# 修改文件/opt/conda/lib/python3.8/site-packages/kubeflow/fairing/builders/cluster/minio_context.py 
# 增加env， http_proxy https_proxy no_proxy
# 特别注意no_proxy需要排除minio-service.kubeflow
#
# client.V1EnvVar(name='http_proxy',
#                                         value='http://172.16.153.24:7890'),
# client.V1EnvVar(name='https_proxy',
#                                         value='http://172.16.153.24:7890'),
# client.V1EnvVar(name='no_proxy',
#                                         value='localhost,127.0.0.1,0.0.0.0,10.0.0.0/8,cattle-system.svc,192.168.10.0/24,.svc,.cluster.local,example.com,localhost,.localhost,127.0.0.0/8,192.168.0.1/16,0.0.0.0,172.0.0.1/8,kubeflow-us-east-1,minio-service.kubeflow'),
# client.V1EnvVar(name='NO_PROXY',
#                                         value='localhost,127.0.0.1,0.0.0.0,10.0.0.0/8,cattle-system.svc,192.168.10.0/24,.svc,.cluster.local,example.com,localhost,.localhost,127.0.0.0/8,192.168.0.1/16,0.0.0.0,172.0.0.1/8,kubeflow-us-east-1,minio-service.kubeflow'),

cluster_builder.build()

[I 220927 00:01:32 cluster:46] Building image using cluster builder.
[I 220927 00:01:32 base:107] Creating docker context: /tmp/fairing_context_tsh3r_tq
[W 220927 00:01:32 base:94] Dockerfile already exists in Fairing context, skipping...
[W 220927 00:01:34 manager:298] Waiting for fairing-builder-5mwcg-8928l to start...
[W 220927 00:01:34 manager:298] Waiting for fairing-builder-5mwcg-8928l to start...
[W 220927 00:01:34 manager:298] Waiting for fairing-builder-5mwcg-8928l to start...
[I 220927 00:01:45 manager:303] Pod started running True


INFO[0000] Retrieving image manifest ubuntu
INFO[0000] Retrieving image ubuntu from registry index.docker.io
INFO[0004] Built cross stage deps: map[]
INFO[0004] Retrieving image manifest ubuntu
INFO[0004] Returning cached image manifest
INFO[0004] Executing 0 build triggers
INFO[0004] Skipping unpacking as no commands require it.
INFO[0004] Taking snapshot of full filesystem...
INFO[0024] ENTRYPOINT ["/bin/bash", "-c", "echo hello"]
INFO[0024] Taking snapshot of full filesystem...
INFO[0044] No files were changed, appending empty layer to config. No layer added to image.
INFO[0044] Pushing image to k3d-myregistry.localhost:5000/mnist:875730ED
INFO[0044] Pushed image to 1 destinations


In [14]:
##https://vectorcloud.io/blog/2021/08/01/%E4%BD%BF%E7%94%A8kaniko%E6%9E%84%E5%BB%BAdocker%E9%95%9C%E5%83%8F/

constants.constants

<module 'kubeflow.fairing.constants.constants' from '/opt/conda/lib/python3.8/site-packages/kubeflow/fairing/constants/constants.py'>

In [1]:
from utils.build_image import build_image

In [3]:
build_image("/home/jovyan/project/pytorch-bert/kaniko","test-image")

[I 220927 00:33:50 cluster:46] Building image using cluster builder.
[I 220927 00:33:50 base:107] Creating docker context: /tmp/fairing_context_aa7oxa5q
[W 220927 00:33:50 base:94] Dockerfile already exists in Fairing context, skipping...


/home/jovyan/project/pytorch-bert/kaniko/pod.yaml --> pod.yaml
/home/jovyan/project/pytorch-bert/kaniko/Dockerfile --> Dockerfile
/home/jovyan/project/pytorch-bert/kaniko/volume.yaml --> volume.yaml
/home/jovyan/project/pytorch-bert/kaniko/volume-claim.yaml --> volume-claim.yaml
/home/jovyan/project/pytorch-bert/kaniko/dockerfile --> dockerfile
/home/jovyan/project/pytorch-bert/kaniko/src --> src
/home/jovyan/project/pytorch-bert/kaniko/src/test.py --> src/test.py


[W 220927 00:33:53 manager:298] Waiting for fairing-builder-xqxpz-xnd8b to start...
[W 220927 00:33:53 manager:298] Waiting for fairing-builder-xqxpz-xnd8b to start...
[W 220927 00:33:53 manager:298] Waiting for fairing-builder-xqxpz-xnd8b to start...
[I 220927 00:34:03 manager:303] Pod started running True


INFO[0000] Retrieving image manifest ubuntu
INFO[0000] Retrieving image ubuntu from registry index.docker.io
INFO[0004] Built cross stage deps: map[]
INFO[0004] Retrieving image manifest ubuntu
INFO[0004] Returning cached image manifest
INFO[0004] Executing 0 build triggers
INFO[0004] Skipping unpacking as no commands require it.
INFO[0004] Taking snapshot of full filesystem...
INFO[0025] ENTRYPOINT ["/bin/bash", "-c", "echo hello"]
INFO[0025] Taking snapshot of full filesystem...
INFO[0045] No files were changed, appending empty layer to config. No layer added to image.
INFO[0045] Pushing image to k3d-myregistry.localhost:5000/test-image:4BDEF49B
INFO[0057] Pushed image to 1 destinations
